<a href="https://colab.research.google.com/github/junkuna/Machine-Learning-Training/blob/main/Cross_Validation_and_Grid_Search_(Wine)_5_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Prevent not to use test sets in the training process
import pandas as pd
wine = pd.read_csv('https://raw.githubusercontent.com/rickiepark/hg-mldl/master/wine.csv')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

## we will make validation set to prevent using many test sets
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42
)

sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42
)

print(sub_input.shape, val_input.shape)

## training using validation sets
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print('The validation sets of traing and test scores are')
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))
print('It is overfitting, we will do 5-fold cross validation sets')

## produce 5-fold cross validation sets and check score
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)

##final score of 5-fold cross validation
import numpy as np
print('The final score of fold cross validation is')
print(np.mean(scores['test_score']))

## split the folds into five cross validation sets to mix cross validation
### cross_validate supports how to split into many folds
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print('The mixed five cross validation score is', np.mean(scores['test_score']))

## for ten cross validation sets
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

## we will find better model using hiper parameter
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

gs.fit(train_input, train_target)

### after the training parameter, it will automatically train the hole train sets
### it is not necessary to make train sets after hiper parameter because of GridSearchCV

dt = gs.best_estimator_
print('Training with hiper parameter of score is ', dt.score(train_input, train_target))
print(gs.best_estimator_)

### check the score of five times training of cross validation sets
print('For each scores of five times training of cross validation sets are')
print(gs.best_params_)
print(gs.cv_results_['mean_test_score'])

best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

### regulate the impurity, tree depth, and minimize node samples
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)
          }

#### let's try using best parameter with 5 cross validation set
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)
print(gs.best_params_)
print(np.max(gs.cv_results_['mean_test_score']))

### We need more details about the scale of parameter settings through 'Random Search'
### It will save more time and large scale for searching the perfact parameter than 'Grid Search'
from scipy.stats import uniform, randint

params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': (2, 25),
          'min_samples_leaf': randint(1, 25),
          }

from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)
print(gs.best_params_)

print(np.max(gs.cv_results_['mean_test_score']))

### the perfact model is in train set already in best_esimator_
print(np.max(gs.cv_results_['mean_test_score']))
dt = gs.best_estimator_
print(dt.score(test_input, test_target))


(4157, 3) (1040, 3)
The validation sets of traing and test scores are
0.9971133028626413
0.864423076923077
It is overfitting, we will do 5-fold cross validation sets
{'fit_time': array([0.05572867, 0.07897329, 0.02334261, 0.02938819, 0.02996111]), 'score_time': array([0.01463747, 0.00222802, 0.00216722, 0.00963736, 0.00209546]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
The final score of fold cross validation is
0.855300214703487
The mixed five cross validation score is 0.855300214703487
0.8574181117533719
Training with hiper parameter of score is  0.9615162593804117
DecisionTreeClassifier(min_impurity_decrease=0.0001, random_state=42)
For each scores of five times training of cross validation sets are
{'min_impurity_decrease': 0.0001}
[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}
{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}
0.8683865773302731
{'max_depth': 46, 'min_imp